In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
import matplotlib.pyplot as plt


In [ ]:

# Load arrays from the saved .npy files
time = np.load("/Users/adaciortan/Desktop/experiment data/daca ai fii un stick/the column/2025-07-01_h2-100bar_SRB-time.npy")
volume_1 = np.load("/Users/adaciortan/Desktop/experiment data/daca ai fii un stick/the column/2025-07-01_h2-100bar_SRB-volume_1.npy")
volume_2 = np.load("/Users/adaciortan/Desktop/experiment data/daca ai fii un stick/the column/2025-07-01_h2-100bar_SRB-volume_2.npy")




In [ ]:
#integrate leak rates if relevant
#integrate leak data
leak_rate1a= 3.12E-07
leak_rate2a= 2.11E-06
trend1a= leak_rate1a * time
trend2a= leak_rate2a * time
volume_1= volume_1 - trend1a
volume_2= volume_2 - trend2a


In [ ]:
def convert_to_moles_and_days(time_array, vol1_array, vol2_array, density, molar_mass):
    """
    Converts Time from minutes to days.
    Converts Volume to Moles using: Volume * (Density / Molar Mass)
    """
    # 1. Convert Time (Minutes -> Days)
    time_days = time_array / 1440.0
    
    # 2. Calculate conversion factor
    # Mass = Volume * Density
    # Moles = Mass / Molar Mass
    conversion_factor = density / molar_mass
    
    # 3. Convert Volumes to Moles
    moles_1 = vol1_array * conversion_factor
    moles_2 = vol2_array * conversion_factor
    
    return time_days, moles_1, moles_2




In [ ]:
#here add density(kg/m^3) and molar mass (g/mol) for density ad different P and T go to https://cmb.tech/hydrogen-tools
#output in days and mmol
time_days,moles_1, moles_2=convert_to_moles_and_days(time,volume_1,volume_2,density=7.397,molar_mass=2.016) 

In [ ]:

# Create the plot
plt.figure(figsize=(8,6))
plt.plot(time_days, moles_1, label="Biotic hydrogen consumption (11.11)", linewidth=1)
plt.plot(time_days, moles_2, label="Sterile control (11.11)", linewidth=1,)

# Labels and legend
plt.xlabel("Days")
plt.ylabel("H2 consumed (mmol)")
plt.title("Hydrogen consumption of SRB")
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)

# Show the plot
plt.show()


In [ ]:

def trim_and_zero_arrays(a, b, c):
    """
    1. Finds the index of the minimum value in array 'a' (ignoring NaNs).
    2. Trims a, b, and c to start from that index.
    3. Shifts 'a' and 'b' so their minimum values become 0 (ignoring NaNs).
    """
    
    # 1. Find the index of the lowest value in 'a'
    # We use nanargmin to ignore the gap we created earlier
    idx = np.nanargmin(a)

    # 2. Slice (Trim) all arrays from that index onwards
    a_trim = a[idx:]
    b_trim = b[idx:]
    c_trim = c[idx:]

    # 3. Shift 'a' and 'b' so they start at 0
    # Use nanmin so the 'gap' doesn't break the subtraction
    a_final = a_trim - np.nanmin(a_trim)
    b_final = b_trim - np.nanmin(b_trim)
    
    c_final = c_trim

    return a_final, b_final, c_final, idx


# Run the function
# We treat 'a_input' as the reference for finding the cut point
moles_2_trimmed, moles_1_trimmed, time_days_trimmed, cut_index = trim_and_zero_arrays(moles_2, moles_1,time_days )



In [2]:
## plot with removed expansion and normalized to zero


# Create the plot
plt.figure(figsize=(10,6))
plt.plot(time_days_trimmed, moles_1_trimmed, label="Biotic hydrogen consumption", linewidth=1,color="green")
plt.plot(time_days_trimmed,moles_2_trimmed, label="Sterile control", linewidth=1,color="red")
#plt.plot(th_cum_min_2, th_c_vol1_2, label="Biotic hydrogen consumption(01.07)", linewidth=1, color="green")
#plt.plot(th_cum_min_2, th_c_vol2_2, label="Sterile control (01.07)", linewidth=1,color="red")
#plt.plot(h_cum_min_1, h_c_vol1_1, label="Volume change with SRB (18.06)", linewidth=1)
#plt.plot(h_cum_min_1, h_c_vol2_1, label="Only Medium (18.06)", linewidth=1,)
# Labels and legend
plt.xlabel("Time (days)")
plt.ylabel("H2 consumed (mmol)")
plt.title("Consumed Hydrogen 400 bar (29.01)") #, Both Experiments
plt.legend()
plt.grid(True, linestyle=":", alpha=0.7)

# Show the plot
plt.show()


NameError: name 'time_days_trimmed' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
moles_consumed=moles_1_trimmed-moles_2_trimmed
time= time_days_trimmed

In [ ]:

plt.figure(figsize=(10,6))
plt.plot(time, moles_consumed, label="Microbial Consumption ", linewidth=1, color="purple")
#plt.plot(th_cum_min_2,th_c_vol2_3, label="Microbial Consumption (01.07)", linewidth=1, color="blue")


plt.xlabel("Time [days]")
plt.ylabel("Consumed Hydrogen [mmol]")
plt.legend()
plt.title("Microbial Consumption 100 bar (01.07) (Biotic-Sterile)")
plt.show()